In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

# Build `states`

In [ ]:
# Imports
from utils import get_pickle, get_order_pf, qualify_conIds, clean_ib_util_df, get_dte, get_margins
import asyncio
import numpy as np
import pandas as pd
from ib_insync import MarketOrder

In [ ]:
# get unds, open orders and portfolio
unds = set(get_pickle(DATAPATH / 'unds.pkl').keys())
df_openorder, df_pf = asyncio.run(get_order_pf(PORT))

In [ ]:
# Get margins of all positions

CID = _vars.CID
desc = "Qualifiying Reap Contracts"
conId_list = df_pf.conId.to_list()

# ... qualify portfolio contracts
pf_contracts = asyncio.run(qualify_conIds(PORT, conId_list, f"Qualifying {MARKET} conIds"))

In [ ]:
# ...integrate df_pf with multiplier

df1 = clean_ib_util_df(pf_contracts).set_index('conId')
df2 = df_pf.set_index('conId')

cols_to_use = df2.columns.difference(df1.columns)
df_pf = df1.join(df2[cols_to_use])

# join the multiplier
s = pd.to_numeric(df_pf.multiplier)
s.fillna(1, inplace=True)
df_pf = df_pf.assign(multiplier=s)

In [ ]:
# Get DTEs
df_pf.insert(4, 'dte', df_pf.expiry.apply(lambda x: get_dte(x, MARKET)))
df_pf.loc[df_pf.dte <=0, "dte"] = 0

# Get the costPrice
df_pf.insert(9, 'costPrice', abs(df_pf.avgCost/df_pf.position))

# Assign the actions
df_pf = df_pf.assign(action=np.where(df_pf.position < 0, "BUY", "SELL"))

In [ ]:
# build the orders
wif_order = [MarketOrder(action, totalQuantity) 
             for action, totalQuantity 
             in zip(df_pf.action, abs(df_pf.position*df_pf.multiplier))]
df_pf = df_pf.assign(wif_order = wif_order)

In [ ]:
from utils import get_margins

In [ ]:
df_pf

In [ ]:
with IB().connect(port=PORT) as ib:
    df_margins = asyncio.run(get_margins(port=PORT, contracts=df_pf.contract, orders = df_pf.wif_order))

In [ ]:
get_pickle(DATAPATH / 'lots.pkl')

In [ ]:
from utils import get_margins
df_pf_m = asyncio.run(get_margins(PORT, df_pf, DATAPATH / 'lots.pkl', 'BUY'))

In [ ]:
df_pf_m.margin.sum()

In [ ]:
async def get_margins(port: int, 
                      contracts: Union[pd.DataFrame, list],
                      orders: Union[MarketOrder, None],                      
                      lots_path: Path=None,
                      ACTION: str='SELL', 
                      chunk_size: int=100,
                      CID: int=0):
    """
    Gets margins for options contracts with `orders` or `ACTION`

    Parameters
    ---
    contracts: df with `contract` field | list
    orders: list of `MarketOrder` | `None` reuqires an `ACTION`
    ACTION: `BUY` or `SELL` needed if no `orders` are provided

    """

    try:
        opt_contracts = contracts.contract.to_list()
    except AttributeError:
        contracts = clean_ib_util_df(contracts)
        opt_contracts = contracts.contract.to_list()


    results = list()

    pbar = tqdm(total=len(opt_contracts),
                    desc="Getting margins:",
                    bar_format = BAR_FORMAT,
                    ncols=80,
                    leave=True,
                )

    contracts = contracts.assign(conId=[c.conId for c in opt_contracts]).\
                set_index('conId')

    chunks = chunk_me(opt_contracts, chunk_size)

    with await IB().connectAsync(port=port, clientId=CID) as ib:
        
        for cts in chunks:

            if orders:

                tasks = [asyncio.create_task(get_a_margin(ib=ib, 
                                                          contract=contract,
                                                          order=orders,
                                                          lots_path=lots_path), 
                                                            name= contract.localSymbol) 
                            for contract in cts]     

            else:
                tasks = [asyncio.create_task(get_a_margin(ib=ib, 
                                                          contract=contract, 
                                                          lots_path=lots_path, 
                                                          ACTION=ACTION), 
                                                            name= contract.localSymbol) 
                            for contract in cts]            


            margin = await asyncio.gather(*tasks)

            results += margin
            pbar.update(len(cts))
            pbar.refresh()

    flat_results ={k: v for r in results for k, v in r.items()}
    df_mgncomm = pd.DataFrame(flat_results).T
    df_out = contracts.join(df_mgncomm).reset_index()

    pbar.close()

    return df_out

## Unsowed

In [ ]:
# No orders to sow and no existing positions

options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'

long = df_pf.position > 0
long_options = long & options
df_pf[long_options]

unsowed = unds - set(df_openorder.symbol) - set(df_pf.symbol)
unsowed

# Orphaned

In [ ]:
# Long calls or puts without any underlying stock position
options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'
long = df_pf.position >= 1
long_stocks = long & stocks
long_options = long & options
orphaned = long_stocks & long_options

orphaned = df_pf[orphaned]
orphaned

## Unreaped

In [ ]:
# Open position without a reap order
df_reap = df_pf[options]